# Vexpresso Walkthrough

## In this walkthrough, we will be using a small dataset of Pokemon statistics to showcase vexpresso's capabilities

### Imports

In [3]:
import json
import pandas as pd
from langchain.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
from vexpresso.collection import Collection
from vexpresso.retrieval import FaissRetrievalStrategy, TopKRetrievalStrategy
from vexpresso.embedding_function import LangChainEmbeddingsFunction

In [4]:
embeddings_fn = LangChainEmbeddingsFunction(HuggingFaceEmbeddings())

## Create Collection
### This may take some time because we are embedding 809 entries

In [5]:
import duckdb



In [6]:
# retrieval_strategy = FaissRetrievalStrategy() # this requires faiss, but is faster than numpy
retrieval_strategy = TopKRetrievalStrategy()

# we can create the content, metadata, ids directly
# content = list(df["description"])
# collection = Collection(
#     content = content,
#     embedding_fn = embeddings_fn,
#     metadata = df,
#     ids = list(df["name"]),
#     retrieval_strategy=retrieval_strategy
# )

# use langchain embeddings_fn
embeddings_fn = LangChainEmbeddingsFunction(HuggingFaceEmbeddings())

# or we can just pass in the string thats located in the metadata
collection = Collection(
    content = "description",
    embeddings_fn = embeddings_fn,
    metadata = "data/pokedex_processed.json",
    ids = "name",
    retrieval_strategy=retrieval_strategy
)

In [98]:
df = collection.df()

In [99]:
df

,id,type,info,profile,name,description,vexpresso_index,content
0,Test,"['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...,0,Seed Pokémon: Bulbasaur can be seen napping in...
1,Ivysaur,"['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...,1,Seed Pokémon: There is a bud on this Pokémon’s...
2,Venusaur,"['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...","{'height': '2 m', 'weight': '100 kg', 'egg': [...",Venusaur,Seed Pokémon: There is a large flower on Venus...,2,Seed Pokémon: There is a large flower on Venus...
3,Charmander,['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",Charmander,Lizard Pokémon: The flame that burns at the ti...,3,Lizard Pokémon: The flame that burns at the ti...
4,Charmeleon,['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...,4,Flame Pokémon: Charmeleon mercilessly destroys...
...,...,...,...,...,...,...,...,...
804,Stakataka,"['Rock', 'Steel']","{'HP': 61, 'Attack': 131, 'Defense': 211, 'Sp....","{'height': '5.5 m', 'weight': '820 kg', 'egg':...",Stakataka,Rampart Pokémon: When stone walls started movi...,804,Rampart Pokémon: When stone walls started movi...
805,Blacephalon,"['Fire', 'Ghost']","{'HP': 53, 'Attack': 127, 'Defense': 53, 'Sp. ...","{'height': '1.8 m', 'weight': '13 kg', 'egg': ...",Blacephalon,Fireworks Pokémon: A UB that appeared from an ...,805,Fireworks Pokémon: A UB that appeared from an ...
806,Zeraora,['Electric'],"{'HP': 88, 'Attack': 112, 'Defense': 75, 'Sp. ...","{'height': '1.5 m', 'weight': '44.5 kg', 'egg'...",Zeraora,Thunderclap Pokémon: It approaches its enemies...,806,Thunderclap Pokémon: It approaches its enemies...
807,Meltan,['Steel'],"{'HP': 46, 'Attack': 65, 'Defense': 65, 'Sp. A...","{'height': '0.2 m', 'weight': '8.0 kg', 'egg':...",Meltan,"Hex Nut Pokémon: They live as a group, but whe...",807,"Hex Nut Pokémon: They live as a group, but whe..."


In [101]:
daft_df = daft.from_pandas(df)

In [95]:
daft_df

idUtf8,typeUtf8,"infoStruct[Attack: Int64, Defense: Int64, HP: Int64, Sp. Attack: Int64, Sp. Defense: Int64, Speed: Int64]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]",nameUtf8,descriptionUtf8,vexpresso_indexInt64,contentUtf8


In [100]:
daft_df.to_pandas()

,id,type,info,profile,name,description,vexpresso_index,content
0,Test,"['Grass', 'Poison']","{'Attack': 49, 'Defense': 49, 'HP': 45, 'Sp. A...","{'ability': [['Overgrow', 'false'], ['Chloroph...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...,0,Seed Pokémon: Bulbasaur can be seen napping in...
1,Ivysaur,"['Grass', 'Poison']","{'Attack': 62, 'Defense': 63, 'HP': 60, 'Sp. A...","{'ability': [['Overgrow', 'false'], ['Chloroph...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...,1,Seed Pokémon: There is a bud on this Pokémon’s...
2,Venusaur,"['Grass', 'Poison']","{'Attack': 82, 'Defense': 83, 'HP': 80, 'Sp. A...","{'ability': [['Overgrow', 'false'], ['Chloroph...",Venusaur,Seed Pokémon: There is a large flower on Venus...,2,Seed Pokémon: There is a large flower on Venus...
3,Charmander,['Fire'],"{'Attack': 52, 'Defense': 43, 'HP': 39, 'Sp. A...","{'ability': [['Blaze', 'false'], ['Solar Power...",Charmander,Lizard Pokémon: The flame that burns at the ti...,3,Lizard Pokémon: The flame that burns at the ti...
4,Charmeleon,['Fire'],"{'Attack': 64, 'Defense': 58, 'HP': 58, 'Sp. A...","{'ability': [['Blaze', 'false'], ['Solar Power...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...,4,Flame Pokémon: Charmeleon mercilessly destroys...
...,...,...,...,...,...,...,...,...
804,Stakataka,"['Rock', 'Steel']","{'Attack': 131, 'Defense': 211, 'HP': 61, 'Sp....","{'ability': [['Beast Boost', 'false']], 'egg':...",Stakataka,Rampart Pokémon: When stone walls started movi...,804,Rampart Pokémon: When stone walls started movi...
805,Blacephalon,"['Fire', 'Ghost']","{'Attack': 127, 'Defense': 53, 'HP': 53, 'Sp. ...","{'ability': [['Beast Boost', 'false']], 'egg':...",Blacephalon,Fireworks Pokémon: A UB that appeared from an ...,805,Fireworks Pokémon: A UB that appeared from an ...
806,Zeraora,['Electric'],"{'Attack': 112, 'Defense': 75, 'HP': 88, 'Sp. ...","{'ability': [['Volt Absorb', 'false']], 'egg':...",Zeraora,Thunderclap Pokémon: It approaches its enemies...,806,Thunderclap Pokémon: It approaches its enemies...
807,Meltan,['Steel'],"{'Attack': 65, 'Defense': 65, 'HP': 46, 'Sp. A...","{'ability': [['Magnet Pull', 'false']], 'egg':...",Meltan,"Hex Nut Pokémon: They live as a group, but whe...",807,"Hex Nut Pokémon: They live as a group, but whe..."


In [84]:
pd.DataFrame({"id":"Fake", "type":["Fake"]})

,id,type
0,Fake,Fake


In [85]:
import pandas as pd

In [96]:
df["id"].iloc[0] = "Test"

In [97]:
daft_df.to_pandas()

,id,type,info,profile,name,description,vexpresso_index,content
0,Test,"['Grass', 'Poison']","{'Attack': 49, 'Defense': 49, 'HP': 45, 'Sp. A...","{'ability': [['Overgrow', 'false'], ['Chloroph...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...,0,Seed Pokémon: Bulbasaur can be seen napping in...
1,Ivysaur,"['Grass', 'Poison']","{'Attack': 62, 'Defense': 63, 'HP': 60, 'Sp. A...","{'ability': [['Overgrow', 'false'], ['Chloroph...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...,1,Seed Pokémon: There is a bud on this Pokémon’s...
2,Venusaur,"['Grass', 'Poison']","{'Attack': 82, 'Defense': 83, 'HP': 80, 'Sp. A...","{'ability': [['Overgrow', 'false'], ['Chloroph...",Venusaur,Seed Pokémon: There is a large flower on Venus...,2,Seed Pokémon: There is a large flower on Venus...
3,Charmander,['Fire'],"{'Attack': 52, 'Defense': 43, 'HP': 39, 'Sp. A...","{'ability': [['Blaze', 'false'], ['Solar Power...",Charmander,Lizard Pokémon: The flame that burns at the ti...,3,Lizard Pokémon: The flame that burns at the ti...
4,Charmeleon,['Fire'],"{'Attack': 64, 'Defense': 58, 'HP': 58, 'Sp. A...","{'ability': [['Blaze', 'false'], ['Solar Power...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...,4,Flame Pokémon: Charmeleon mercilessly destroys...
...,...,...,...,...,...,...,...,...
804,Stakataka,"['Rock', 'Steel']","{'Attack': 131, 'Defense': 211, 'HP': 61, 'Sp....","{'ability': [['Beast Boost', 'false']], 'egg':...",Stakataka,Rampart Pokémon: When stone walls started movi...,804,Rampart Pokémon: When stone walls started movi...
805,Blacephalon,"['Fire', 'Ghost']","{'Attack': 127, 'Defense': 53, 'HP': 53, 'Sp. ...","{'ability': [['Beast Boost', 'false']], 'egg':...",Blacephalon,Fireworks Pokémon: A UB that appeared from an ...,805,Fireworks Pokémon: A UB that appeared from an ...
806,Zeraora,['Electric'],"{'Attack': 112, 'Defense': 75, 'HP': 88, 'Sp. ...","{'ability': [['Volt Absorb', 'false']], 'egg':...",Zeraora,Thunderclap Pokémon: It approaches its enemies...,806,Thunderclap Pokémon: It approaches its enemies...
807,Meltan,['Steel'],"{'Attack': 65, 'Defense': 65, 'HP': 46, 'Sp. A...","{'ability': [['Magnet Pull', 'false']], 'egg':...",Meltan,"Hex Nut Pokémon: They live as a group, but whe...",807,"Hex Nut Pokémon: They live as a group, but whe..."


In [76]:
content = ["FakePokemon: this is a fake"]
metadata = {"id":"Fake", "type":["Fake"]}


In [8]:
db = duckdb.from_df(df)

In [10]:
import daft

In [37]:
daft_df = daft.from_pandas(df)

In [20]:
type(daft_df.to_pandas())

pandas.core.frame.DataFrame

In [31]:
embeddings = collection.embeddings.raw_embeddings

In [32]:
embeddings

array([[ 0.01649672, -0.06166817, -0.02860884, ..., -0.00741717,
         0.03515032, -0.04716535],
       [ 0.005456  , -0.0296776 , -0.02466376, ...,  0.01295663,
         0.00113479, -0.04750562],
       [ 0.03115942, -0.03395105, -0.0280442 , ...,  0.03063581,
         0.04841576, -0.02030743],
       ...,
       [ 0.03435412, -0.05174266, -0.00161358, ..., -0.00561018,
         0.03141578, -0.0134402 ],
       [ 0.05148792,  0.00510018,  0.00752965, ...,  0.03269063,
         0.0799771 , -0.01039558],
       [ 0.02068185, -0.06435209, -0.00053274, ..., -0.03226795,
         0.00670514,  0.0106067 ]])

In [68]:
embeddings = np.zeros((100, 3, 64, 64))

In [60]:
embeddings_df = daft.from_pydict({"embeddings":embeddings})

In [71]:
embeddings_df.to_pandas()

,embeddings
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...
95,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
96,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
97,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
98,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [72]:
pd.DataFrame({"embeddings":[embeddings[i] for i in range(embeddings.shape[0])]})

,embeddings
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...
95,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
96,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
97,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
98,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [73]:
df = daft.from_glob_path("s3://daft-public-data/open-images/validation-images/*")


2023-05-19 22:57:08.014 | ERROR    | daft.filesystem:_get_s3fs_kwargs:37 - Error when importing botocore. Install getdaft[aws] for the required 3rd party dependencies to interact with AWS S3 (https://www.getdaft.io/projects/docs/en/latest/learn/install.html)


ModuleNotFoundError: No module named 'botocore'

In [56]:
embeddings_df.to_pandas().iloc[0].dot(embeddings_df.to_pandas().iloc[0])

array([2.72141825e-04, 3.80296350e-03, 8.18465511e-04, 9.86501356e-04,
       1.33378091e-03, 1.30986295e-04, 1.60707909e-04, 2.64239033e-04,
       1.07249901e-02, 2.47406564e-04, 1.46148292e-03, 6.11956237e-05,
       2.51511852e-04, 3.93538177e-04, 3.34183255e-03, 6.45527071e-03,
       5.48195411e-08, 3.81825458e-04, 2.65715869e-06, 3.52731957e-05,
       9.98482210e-06, 3.74263917e-05, 5.49642301e-04, 2.94458770e-04,
       3.77443214e-03, 2.37156239e-03, 1.92771684e-03, 9.64023057e-04,
       1.23858422e-03, 1.90206336e-04, 1.61133954e-04, 3.38517473e-03,
       3.46009270e-04, 8.20242100e-05, 3.27658014e-12, 1.16868401e-04,
       5.00522966e-04, 2.75044267e-03, 3.63554632e-04, 1.12622074e-05,
       6.57550286e-04, 2.10663500e-04, 1.77316769e-03, 2.91365403e-05,
       7.83748895e-04, 1.90197856e-03, 1.33475522e-03, 3.79713590e-04,
       2.42250685e-04, 4.40015003e-06, 3.45354652e-05, 2.46588191e-04,
       3.31216791e-05, 4.38284477e-04, 7.14622463e-03, 9.64799560e-04,
      

In [43]:
daft_df.join(embeddings_df)

ValueError: DaftError::NotFound Field: embeddings not found in [Field { name: "id", dtype: Utf8 }, Field { name: "type", dtype: Utf8 }, Field { name: "info", dtype: Struct([Field { name: "Attack", dtype: Int64 }, Field { name: "Defense", dtype: Int64 }, Field { name: "HP", dtype: Int64 }, Field { name: "Sp. Attack", dtype: Int64 }, Field { name: "Sp. Defense", dtype: Int64 }, Field { name: "Speed", dtype: Int64 }]) }, Field { name: "profile", dtype: Struct([Field { name: "ability", dtype: List(Field { name: "item", dtype: List(Field { name: "item", dtype: Utf8 }) }) }, Field { name: "egg", dtype: List(Field { name: "item", dtype: Utf8 }) }, Field { name: "gender", dtype: Utf8 }, Field { name: "height", dtype: Utf8 }, Field { name: "weight", dtype: Utf8 }]) }, Field { name: "name", dtype: Utf8 }, Field { name: "description", dtype: Utf8 }, Field { name: "vexpresso_index", dtype: Int64 }, Field { name: "content", dtype: Utf8 }]

In [33]:
daft_df.with_column("embeddings", embeddings)

APITypeError: DataFrame.with_column received wrong input type.
Required:
	expr = <<class 'daft.expressions.expressions.Expression'>>
Given:
	expr = <ndarray>

In [29]:
daft_df.select("name", "info")

nameUtf8,"infoStruct[Attack: Int64, Defense: Int64, HP: Int64, Sp. Attack: Int64, Sp. Defense: Int64, Speed: Int64]"


In [15]:
db = duckdb.from_df(daft_df.collect())

TypeError: from_df(): incompatible function arguments. The following argument types are supported:
    1. (df: pandas.DataFrame = None, connection: duckdb.DuckDBPyConnection = None) -> duckdb.DuckDBPyRelation
    2. (df: pandas.DataFrame, connection: duckdb.DuckDBPyConnection = None) -> duckdb.DuckDBPyRelation

Invoked with: +------------+---------------------+---------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+------------+----------------------+-------------------+----------------------+
| id         | type                | info                                                                                                    | profile                                                                                        | name       | description          |   vexpresso_index | content              |
| Utf8       | Utf8                

In [4]:
collection.df().head(10)

,id,type,info,profile,name,description,vexpresso_index,content
0,Bulbasaur,"['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...,0,Seed Pokémon: Bulbasaur can be seen napping in...
1,Ivysaur,"['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...,1,Seed Pokémon: There is a bud on this Pokémon’s...
2,Venusaur,"['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...","{'height': '2 m', 'weight': '100 kg', 'egg': [...",Venusaur,Seed Pokémon: There is a large flower on Venus...,2,Seed Pokémon: There is a large flower on Venus...
3,Charmander,['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",Charmander,Lizard Pokémon: The flame that burns at the ti...,3,Lizard Pokémon: The flame that burns at the ti...
4,Charmeleon,['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...,4,Flame Pokémon: Charmeleon mercilessly destroys...
5,Charizard,"['Fire', 'Flying']","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",Charizard,Flame Pokémon: Charizard flies around the sky ...,5,Flame Pokémon: Charizard flies around the sky ...
6,Squirtle,['Water'],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",Squirtle,Tiny Turtle Pokémon: Squirtle’s shell is not m...,6,Tiny Turtle Pokémon: Squirtle’s shell is not m...
7,Wartortle,['Water'],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",Wartortle,Turtle Pokémon: Its tail is large and covered ...,7,Turtle Pokémon: Its tail is large and covered ...
8,Blastoise,['Water'],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",Blastoise,Shellfish Pokémon: Blastoise has water spouts ...,8,Shellfish Pokémon: Blastoise has water spouts ...
9,Caterpie,['Bug'],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",Caterpie,Worm Pokémon: Its body is soft and weak. In na...,9,Worm Pokémon: Its body is soft and weak. In na...


### you can also create this without metadata or ids, or even just embeddings

```python
without_metadata = Collection(
    content,
    embedding_fn = embeddings_fn,
    retrieval_strategy=retrieval_strategy
)

embeddings = embeddings_fn.batch_embed(content)

without_embeddings = Collection(
    embeddings,
    embedding_fn = embeddings_fn,
    retrieval_strategy=retrieval_strategy
)

```

## Query by Pokemon description

### Let's find a pokemon that "loves to sleep"

In [8]:
query = "Loves to sleep"
sleepy_pokemon = collection.query("Loves to sleep", k=10)

In [9]:
sleepy_pokemon.df()

,id,type,info,profile,name,description,vexpresso_index,content
142,Snorlax,['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....","{'height': '2.1 m', 'weight': '460 kg', 'egg':...",Snorlax,Sleeping Pokémon: It has no interest in anythi...,0,Sleeping Pokémon: It has no interest in anythi...
660,Fletchling,"['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...","{'height': '0.3 m', 'weight': '1.7 kg', 'egg':...",Fletchling,Tiny Robin Pokémon: Its body is always warm. T...,1,Tiny Robin Pokémon: Its body is always warm. T...
774,Komala,['Normal'],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. ...","{'height': '0.4 m', 'weight': '19.9 kg', 'egg'...",Komala,Drowsing Pokémon: It remains asleep from birth...,2,Drowsing Pokémon: It remains asleep from birth...
160,Sentret,['Normal'],"{'HP': 35, 'Attack': 46, 'Defense': 34, 'Sp. A...","{'height': '0.8 m', 'weight': '6 kg', 'egg': [...",Sentret,"Scout Pokémon: When Sentret sleeps, it does so...",3,"Scout Pokémon: When Sentret sleeps, it does so..."
300,Delcatty,['Normal'],"{'HP': 70, 'Attack': 65, 'Defense': 65, 'Sp. A...","{'height': '1.1 m', 'weight': '32.6 kg', 'egg'...",Delcatty,Prim Pokémon: Delcatty sleeps anywhere it want...,4,Prim Pokémon: Delcatty sleeps anywhere it want...
69,Weepinbell,"['Grass', 'Poison']","{'HP': 65, 'Attack': 90, 'Defense': 50, 'Sp. A...","{'height': '1 m', 'weight': '6.4 kg', 'egg': [...",Weepinbell,Flycatcher Pokémon: Weepinbell has a large hoo...,5,Flycatcher Pokémon: Weepinbell has a large hoo...
703,Goomy,['Dragon'],"{'HP': 45, 'Attack': 50, 'Defense': 35, 'Sp. A...","{'height': '0.3 m', 'weight': '2.8 kg', 'egg':...",Goomy,Soft Tissue Pokémon: Its body is mostly water....,6,Soft Tissue Pokémon: Its body is mostly water....
95,Drowzee,['Psychic'],"{'HP': 60, 'Attack': 48, 'Defense': 45, 'Sp. A...","{'height': '1 m', 'weight': '32.4 kg', 'egg': ...",Drowzee,Hypnosis Pokémon: It puts its prey to sleep an...,7,Hypnosis Pokémon: It puts its prey to sleep an...
490,Darkrai,['Dark'],"{'HP': 70, 'Attack': 90, 'Defense': 90, 'Sp. A...","{'height': '1.5 m', 'weight': '50.5 kg', 'egg'...",Darkrai,Pitch-Black Pokémon: It can lull people to sle...,8,Pitch-Black Pokémon: It can lull people to sle...
24,Pikachu,['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. A...","{'height': '0.4 m', 'weight': '6 kg', 'egg': [...",Pikachu,"Mouse Pokémon: While sleeping, it generates el...",9,"Mouse Pokémon: While sleeping, it generates el..."


In [14]:
ghost_sleepy_pokemon = sleepy_pokemon.query("Ghost Pokemon, very spooky", k=1)

In [15]:
ghost_sleepy_pokemon.df()

,id,type,info,profile,name,description,vexpresso_index,content
490,Darkrai,['Dark'],"{'HP': 70, 'Attack': 90, 'Defense': 90, 'Sp. A...","{'height': '1.5 m', 'weight': '50.5 kg', 'egg'...",Darkrai,Pitch-Black Pokémon: It can lull people to sle...,0,Pitch-Black Pokémon: It can lull people to sle...


## Grabbing metadata fields

In [7]:
# get a single field
names = sleepy_pokemon.get_field("name")
types = sleepy_pokemon.get_field("type")
descriptions = sleepy_pokemon.get_field("description")

# get multiple
names, types, descriptions = sleepy_pokemon.get_fields(["name", "type", "description"])
for n, t, d in zip(names, types, descriptions):
    print(f"{n}: {t} -- {d}")

Snorlax: ['Normal'] -- Sleeping Pokémon: It has no interest in anything other than eating. Even if you climb up on its stomach while it’s napping, it doesn’t seem to mind at all!
Fletchling: ['Normal', 'Flying'] -- Tiny Robin Pokémon: Its body is always warm. Trainers who live in cold areas apparently sleep with it in their bed.
Komala: ['Normal'] -- Drowsing Pokémon: It remains asleep from birth to death as a result of the sedative properties of the leaves that form its diet.
Sentret: ['Normal'] -- Scout Pokémon: When Sentret sleeps, it does so while another stands guard. The sentry wakes the others at the first sign of danger. When this Pokémon becomes separated from its pack, it becomes incapable of sleep due to fear.
Delcatty: ['Normal'] -- Prim Pokémon: Delcatty sleeps anywhere it wants without keeping a permanent nest. If other Pokémon approach it as it sleeps, this Pokémon will never fight—it will just move away somewhere else.
Weepinbell: ['Grass', 'Poison'] -- Flycatcher Pokém

## Performing filters

### With vexpresso we can leverage duckdb for powerful queries

#### In order to filter on metadata, you must supply a filter dictionary to the filter method. The dictionary must have the following structure:

```python
{
    <field>: {
        <filter_method>: <value>
    },
    <field>: {
        <filter_method>: <value>
    },
}

```

#### An example query to only get Pokemon with the name "Snorlax":


In [8]:
filter_condition = {
    "name": {
        "eq":"Snorlax"
    }
}

In [9]:
sleepy_pokemon.filter(filter_condition).df()

,id,type,info,profile,name,description,vexpresso_index,content
142,Snorlax,['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....","{'height': '2.1 m', 'weight': '460 kg', 'egg':...",Snorlax,Sleeping Pokémon: It has no interest in anythi...,0,Sleeping Pokémon: It has no interest in anythi...


#### Here are the supported filter methods:

In [10]:
sleepy_pokemon.metadata.print_filter_methods()

contains: 
        {field} (str) contains {value} (str)
        
----------------------------------
eq: 
        {field} equal to {value} (str, int, float)
        
----------------------------------
gt: 
        {field} greater than {value} (int, float)
        
----------------------------------
gte: 
        {field} greater than or equal to {value} (int, float)
        
----------------------------------
isin: 
        {field} is in list of {values} (list of str, int, or float)
        
----------------------------------
lt: 
        {field} less than {value} (int, float)
        
----------------------------------
lte: 
        {field} less than or equal to {value} (int, float)
        
----------------------------------
neq: 
        {field} not equal to {value} (str, int, float)
        
----------------------------------
notcontains: 
        {field} (str) does not contains {value} (str)
        
----------------------------------
notin: 
        {field} not in list of {values} 

#### Let's filter the above collection for "Normal" type Pokemon

In [11]:
filter_condition = {"type":{"contains":"Normal"}}
normal_sleepy_pokemon = sleepy_pokemon.filter(filter_condition)

In [12]:
normal_sleepy_pokemon.df()

,id,type,info,profile,name,description,vexpresso_index,content
142,Snorlax,['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....","{'height': '2.1 m', 'weight': '460 kg', 'egg':...",Snorlax,Sleeping Pokémon: It has no interest in anythi...,0,Sleeping Pokémon: It has no interest in anythi...
660,Fletchling,"['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...","{'height': '0.3 m', 'weight': '1.7 kg', 'egg':...",Fletchling,Tiny Robin Pokémon: Its body is always warm. T...,1,Tiny Robin Pokémon: Its body is always warm. T...
774,Komala,['Normal'],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. ...","{'height': '0.4 m', 'weight': '19.9 kg', 'egg'...",Komala,Drowsing Pokémon: It remains asleep from birth...,2,Drowsing Pokémon: It remains asleep from birth...
160,Sentret,['Normal'],"{'HP': 35, 'Attack': 46, 'Defense': 34, 'Sp. A...","{'height': '0.8 m', 'weight': '6 kg', 'egg': [...",Sentret,"Scout Pokémon: When Sentret sleeps, it does so...",3,"Scout Pokémon: When Sentret sleeps, it does so..."
300,Delcatty,['Normal'],"{'HP': 70, 'Attack': 65, 'Defense': 65, 'Sp. A...","{'height': '1.1 m', 'weight': '32.6 kg', 'egg'...",Delcatty,Prim Pokémon: Delcatty sleeps anywhere it want...,4,Prim Pokémon: Delcatty sleeps anywhere it want...


### We can also filter for multiple conditions on nested data

#### For example, let's filter the above collection for Pokemon with HP >= 35 and Defense < 45

In [13]:
filter_condition = {"info.HP": {"gte":35}, "info.Defense": {"lt":45}}
filtered = normal_sleepy_pokemon.filter(filter_condition)

In [14]:
filtered.df()

,id,type,info,profile,name,description,vexpresso_index,content
660,Fletchling,"['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...","{'height': '0.3 m', 'weight': '1.7 kg', 'egg':...",Fletchling,Tiny Robin Pokémon: Its body is always warm. T...,0,Tiny Robin Pokémon: Its body is always warm. T...
160,Sentret,['Normal'],"{'HP': 35, 'Attack': 46, 'Defense': 34, 'Sp. A...","{'height': '0.8 m', 'weight': '6 kg', 'egg': [...",Sentret,"Scout Pokémon: When Sentret sleeps, it does so...",1,"Scout Pokémon: When Sentret sleeps, it does so..."


### Adding and removing data

#### Add a fake Pokemon

In [15]:
content = ["FakePokemon: this is a fake"]
metadata = {"id":"Fake", "type":["Fake"]}

filtered = filtered.add(content=content, metadata=metadata)

In [16]:
filtered.df()

,id,type,info,profile,name,description,vexpresso_index,content
0,Fletchling,"['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...","{'height': '0.3 m', 'weight': '1.7 kg', 'egg':...",Fletchling,Tiny Robin Pokémon: Its body is always warm. T...,0,Tiny Robin Pokémon: Its body is always warm. T...
1,Sentret,['Normal'],"{'HP': 35, 'Attack': 46, 'Defense': 34, 'Sp. A...","{'height': '0.8 m', 'weight': '6 kg', 'egg': [...",Sentret,"Scout Pokémon: When Sentret sleeps, it does so...",1,"Scout Pokémon: When Sentret sleeps, it does so..."
2,Fake,[Fake],null,null,null,null,2,FakePokemon: this is a fake


#### Remove it

In [17]:
filtered = filtered.remove(ids="Fake")

In [18]:
filtered.df()

,id,type,info,profile,name,description,vexpresso_index,content
0,Fletchling,"['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...","{'height': '0.3 m', 'weight': '1.7 kg', 'egg':...",Fletchling,Tiny Robin Pokémon: Its body is always warm. T...,0,Tiny Robin Pokémon: Its body is always warm. T...
1,Sentret,['Normal'],"{'HP': 35, 'Attack': 46, 'Defense': 34, 'Sp. A...","{'height': '0.8 m', 'weight': '6 kg', 'egg': [...",Sentret,"Scout Pokémon: When Sentret sleeps, it does so...",1,"Scout Pokémon: When Sentret sleeps, it does so..."


## Save collection

In [19]:
filtered.save("./filtered_pokemon")

saving to ./filtered_pokemon


'./filtered_pokemon'

## Load collection

In [20]:
loaded = Collection.from_saved("./filtered_pokemon", embeddings_fn=embeddings_fn)

In [21]:
loaded.df()

,id,type,info,profile,name,description,vexpresso_index,content
0,Fletchling,"['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...","{'height': '0.3 m', 'weight': '1.7 kg', 'egg':...",Fletchling,Tiny Robin Pokémon: Its body is always warm. T...,0,Tiny Robin Pokémon: Its body is always warm. T...
1,Sentret,['Normal'],"{'HP': 35, 'Attack': 46, 'Defense': 34, 'Sp. A...","{'height': '0.8 m', 'weight': '6 kg', 'egg': [...",Sentret,"Scout Pokémon: When Sentret sleeps, it does so...",1,"Scout Pokémon: When Sentret sleeps, it does so..."


## Uploading to HFhub

In [22]:
collection.df().head(10)

,id,type,info,profile,name,description,vexpresso_index,content
0,Bulbasaur,"['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...,0,Seed Pokémon: Bulbasaur can be seen napping in...
1,Ivysaur,"['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...,1,Seed Pokémon: There is a bud on this Pokémon’s...
2,Venusaur,"['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...","{'height': '2 m', 'weight': '100 kg', 'egg': [...",Venusaur,Seed Pokémon: There is a large flower on Venus...,2,Seed Pokémon: There is a large flower on Venus...
3,Charmander,['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",Charmander,Lizard Pokémon: The flame that burns at the ti...,3,Lizard Pokémon: The flame that burns at the ti...
4,Charmeleon,['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...,4,Flame Pokémon: Charmeleon mercilessly destroys...
5,Charizard,"['Fire', 'Flying']","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",Charizard,Flame Pokémon: Charizard flies around the sky ...,5,Flame Pokémon: Charizard flies around the sky ...
6,Squirtle,['Water'],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",Squirtle,Tiny Turtle Pokémon: Squirtle’s shell is not m...,6,Tiny Turtle Pokémon: Squirtle’s shell is not m...
7,Wartortle,['Water'],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",Wartortle,Turtle Pokémon: Its tail is large and covered ...,7,Turtle Pokémon: Its tail is large and covered ...
8,Blastoise,['Water'],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",Blastoise,Shellfish Pokémon: Blastoise has water spouts ...,8,Shellfish Pokémon: Blastoise has water spouts ...
9,Caterpie,['Bug'],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",Caterpie,Worm Pokémon: Its body is soft and weak. In na...,9,Worm Pokémon: Its body is soft and weak. In na...


In [23]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via collection.save(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"

collection.save(repo_id, to_hub=True, private=True)


Uploading collection to shyamsn97/pokemon_vexpresso_test
Upload to shyamsn97/pokemon_vexpresso_test complete!


'shyamsn97/pokemon_vexpresso_test'

In [24]:
# or upload with username + repo name
collection.save(hf_username = "shyamsn97", repo_name = "pokemon_vexpresso_test", to_hub=True)

Uploading collection to None
Upload to shyamsn97/pokemon_vexpresso_test complete!


'shyamsn97/pokemon_vexpresso_test'

## Load from HfHub

In [25]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via Collection.load(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"

collection = Collection.load(repo_id, embeddings_fn = embeddings_fn)

Retrieving from hf repo: shyamsn97/pokemon_vexpresso_test


Fetching 4 files: 100%|███████████████████████████████████████████████████| 4/4 [00:00<00:00, 16.63it/s]


In [26]:
# or load with username + repo name
collection = Collection.load(hf_username = "shyamsn97", repo_name = "pokemon_vexpresso_test", embeddings_fn = embeddings_fn)

Retrieving from hf repo: shyamsn97/pokemon_vexpresso_test


Fetching 4 files: 100%|█████████████████████████████████████████████████| 4/4 [00:00<00:00, 2055.53it/s]


In [27]:
collection.df().head(10)

,id,type,info,profile,name,description,vexpresso_index,content
0,Bulbasaur,"['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",Bulbasaur,Seed Pokémon: Bulbasaur can be seen napping in...,0,Seed Pokémon: Bulbasaur can be seen napping in...
1,Ivysaur,"['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",Ivysaur,Seed Pokémon: There is a bud on this Pokémon’s...,1,Seed Pokémon: There is a bud on this Pokémon’s...
2,Venusaur,"['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...","{'height': '2 m', 'weight': '100 kg', 'egg': [...",Venusaur,Seed Pokémon: There is a large flower on Venus...,2,Seed Pokémon: There is a large flower on Venus...
3,Charmander,['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",Charmander,Lizard Pokémon: The flame that burns at the ti...,3,Lizard Pokémon: The flame that burns at the ti...
4,Charmeleon,['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",Charmeleon,Flame Pokémon: Charmeleon mercilessly destroys...,4,Flame Pokémon: Charmeleon mercilessly destroys...
5,Charizard,"['Fire', 'Flying']","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",Charizard,Flame Pokémon: Charizard flies around the sky ...,5,Flame Pokémon: Charizard flies around the sky ...
6,Squirtle,['Water'],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",Squirtle,Tiny Turtle Pokémon: Squirtle’s shell is not m...,6,Tiny Turtle Pokémon: Squirtle’s shell is not m...
7,Wartortle,['Water'],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",Wartortle,Turtle Pokémon: Its tail is large and covered ...,7,Turtle Pokémon: Its tail is large and covered ...
8,Blastoise,['Water'],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",Blastoise,Shellfish Pokémon: Blastoise has water spouts ...,8,Shellfish Pokémon: Blastoise has water spouts ...
9,Caterpie,['Bug'],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",Caterpie,Worm Pokémon: Its body is soft and weak. In na...,9,Worm Pokémon: Its body is soft and weak. In na...
